# week9 - Tutorial - 
### Working with mod files, defining ion channels in your simulation

In this tutorial we will see how to build mod files (files with channel information behaviour)  

__The first thing you have to do__ is to go to the MOODLE and __download all the files in the tutorial folder__.

If you take a look to the file names, you will see that there are some with the string __'withGaps'__.
The mod files can be opened with a text editor.

As you can imagine, you have to __fill in these gaps__ before using the files for the tutorial.

You will need the __papers__ that are already in the folder from the MOODLE and you can take a look to the other files 
as examples.

Go for it!!

Once you have all the mod files completed you can go on with the tutorial.

__Upload__ the mod files into the storage folder. 
It could be a good idea to create a folder, for example called '/modFiles_week9/' 
and safe the files in it.

In [ ]:
 # To avoid problems, install the pacakges you will need.
! pip install --upgrade pip
! pip install mpmath
! pip install json2html
! pip install -q --upgrade "hbp-service-client==1.0.0"

### RESTART YOUR KERNEL NOW!

In [ ]:
# Download the MOD files from the Storage to your collab virtual machine
# Be patient it could take some minutes...

# These lines allow us to take and work with the desired files in the storage.
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
data_dir = collab_path + '/modFiles_week9/'

#print files_list to check if all of them have been properly downloaded
project_uuid = clients.storage.api_client.get_entity_by_query(path=data_dir)['uuid']
l = clients.storage.api_client.list_folder_content(project_uuid, page_size=20)

filenames = []
for e in l['results']:
    filenames.append(e['name'])

for filename in filenames:
    print filename
    path_to_file = 'downloaded_'+filename
    file_from_path = data_dir+filename
    clients.storage.download_file(str(data_dir+filename), str(path_to_file))

In [ ]:
# First we check if the files are where they suppose to be
! ls

In [ ]:
# This will compile the mechanisms (and create a new folder x86_64/special).
! nrnivmodl

### RESTART YOUR KERNEL NOW!

In [ ]:
# In case you want to check how this neuron looks like, run this cell
import neurom
import neurom.viewer
%matplotlib inline
fig, ax = neurom.viewer.draw(neurom.load_neuron('downloaded_dend-C060114A2_axon-C060114A5.asc'))

__Exercise__: Complete the code below.

The code is based in the one from __week 3__, you can use it to help you to do the exercise

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)

# Import NEURON as a Python module, together with the hoc language
from neuron import h, gui
import neurom as nm

class PCL5_neuron:
    """
    Model inspired from a thalamo-cortical neuron (week 3 ISN course). 
    """
    def __init__(self):
        self.build_morphology()
        self.define_biophysics()
    
    def build_morphology(self):
        """
        Loads a 3D morphology (without axon) of the neuron
        """
        # Load hoc routines to import 3D morphologies
        h.load_file("import3d.hoc")
        nl = h.Import3d_Neurolucida3()  # We have an ascii morphology file
        
        # Read the file and creates automatically section.connect(parent) statements
        nl.input('downloaded_dend-C060114A2_axon-C060114A5.asc')

        # instantiate morphology for simulation
        self.importedcell = h.Import3d_GUI(nl,0)
        self.importedcell.instantiate(None) # Executes the connect statements and loads the cell into h scope

        # Create python lists from the morphology
        self.somatic = []
        self.dendritic = []
        self.all = []
        for sec in h.allsec():
            if 'soma' in sec.name():
                self.somatic.append(sec)
            if 'dend' in sec.name():
                self.dendritic.append(sec)
        
    def define_biophysics(self):
        """
        Distributes passive mechanisms and the different types
        of ion channels on the soma and dendrites
        """
        # Define passive properties
        # You can try to change the passice properties and see what happens
        for sec in self.all:
            sec.insert("pas")
            sec.g_pas = 0.003
            sec.e_pas = -65
            sec.ra = 173
            sec.cm = 1
            sec.ek = -90
            sec.ena = 55
            
        for sec in self.somatic:
            # sec.insert('SUFFIX from mod file')    
            ...
            ...
            ...
        for sec in self.dendritic:
            # sec.insert('SUFFIX from mod file')
            ...
            ...
            ...
            
            

__Exercise__: Play with your simulation, stimulate your neuron with different protocols created by you
and plot the result.

In [ ]:
# Run current clamp experiments with different protocols of your choice
# You can take protocol examples from the previous exercises.

# Instantiate a cell from the class PCL5_neuron
cell = PCL5_neuron()

# Create a dictionary with neuron vectors 'time' and 'v_soma'
# to save the values during the simulation
recordings = {}
recordings['time'] = h.Vector()
recordings['v_soma'] = h.Vector()

recordings['time'].record(h._ref_t)
recordings['v_soma'].record(cell.somatic[0](0.5)._ref_v)

iclamp = h.IClamp(0.5, cell.somatic[0])
iclamp.delay = ... # starting
iclamp.amp = ... # step current
iclamp.dur = ... # duration

# Initialize and run a simulation
h.v_init = -65
h.finitialize()
h.celsius = ... # experiment temperature
h.tstop = ... # experiment duration
h.steps_per_ms = ... # sampling time step

h.init()
h.run()

# plot the result
plt.plot(recordings['time'], recordings['v_soma'])
